In [1]:
#!pip install newspaper3k

In [2]:
#!pip install nltk

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#imports important libraries
import pandas as pd
import numpy as np
import csv
import pickle
from newspaper import Article
import joblib
import time

In [5]:
#load the links

df = pd.read_csv('capital-news.csv', index_col=0)
df.head()

,link_name
num,
1,https://www.capitalfm.co.ke/news/2020/01/locus...
2,https://www.capitalfm.co.ke/business/2020/04/c...
3,https://www.capitalfm.co.ke/news/2020/01/locus...
4,https://www.capitalfm.co.ke/news/2020/01/churc...
5,https://www.capitalfm.co.ke/news/2020/02/deser...


In [6]:
# creating a list of the links
links = []
for i in df['link_name']:
    links.append(i)

In [7]:
links

['https://www.capitalfm.co.ke/news/2020/01/locusts-invade-tseikuru-in-kitui-days-after-a-swarm-was-sighted-in-baringo/',
 'https://www.capitalfm.co.ke/business/2020/04/covid-19-locusts-invasion-to-slow-down-kenyas-gdp-growth-to-3-5pc/',
 'https://www.capitalfm.co.ke/news/2020/01/locust-invasion-in-kitui-machakos-spells-doom-for-farmers-in-eastern-region/',
 'https://www.capitalfm.co.ke/news/2020/01/church-leader-urges-govt-to-partner-with-the-world-in-fighting-locust-invasion/',
 'https://www.capitalfm.co.ke/news/2020/02/desert-locusts-invade-kisumus-muhoroni-unsettling-farmers/',
 'https://www.capitalfm.co.ke/news/2019/12/leaders-appeal-for-help-after-locusts-invade-farms-in-north-kenya/',
 'https://www.capitalfm.co.ke/news/2020/05/munya-says-desert-locust-invasion-in-kenya-has-been-contained/',
 'https://www.capitalfm.co.ke/business/2020/01/swarms-of-locusts-could-grow-by-500-times-by-june-if-not-controlled-oxfam/',
 'https://www.capitalfm.co.ke/news/2020/03/experts-to-file-locust-in

In [8]:

start = time.time() #for code profiling   

 #Intialize list articles_info list
articles_data = []

for a in links:
    #Intialize dictionary
    article_dict = {}
    #Insert link "i" into the dictionary
    article_dict["link"] = a

    article = Article(a)
    article.download()

    #allow 1 sec to download then parse
    time.sleep(1)

    #Try/except is included because not all articles can be parsable   
    try:
        #If article is parsable, create keys
        article.parse()
        article_dict["text"] = article.text
        article_dict["title"] = article.title
        article_dict["date"] = article.publish_date
        article.nlp()
        article_dict["keywords"] = article.keywords
        article_dict["summary"] = article.summary

    except: #newspaper.article.ArticelException:
        pass
        #If article is not parsable create nans
        article_dict["text"] = np.nan
        article_dict["title"] = np.nan
        article_dict["date"] = np.nan
        article_dict["keywords"] = np.nan
        article_dict["summary"] = np.nan


    #Insert dictionary of articles into the articles_data list
    articles_data.append(article_dict)

#create a df
df = pd.DataFrame(articles_data)

#check the time spent to run the process
print("Script took {:.2f} seconds to complete".format(time.time() - start))

Script took 119.66 seconds to complete


In [9]:
df.head()

,link,text,title,date,keywords,summary
0,https://www.capitalfm.co.ke/news/2020/01/locus...,"72 SHARES Share Tweet\n\nKITUI, Kenya, Jan 21 ...",Locusts invade Tseikuru in Kitui days after a ...,2020-01-21 15:38:31+00:00,"[kitui, sighted, spread, latest, tseikuru, bar...","72 SHARES Share TweetKITUI, Kenya, Jan 21 – Ne..."
1,https://www.capitalfm.co.ke/business/2020/04/c...,"5 SHARES Share Tweet\n\nNAIROBI, Kenya, Apr 20...","COVID-19, locusts invasion to slow down Kenya’...",2020-04-20 08:26:09+00:00,"[countrys, kenyas, expect, credit, deficit, fo...","5 SHARES Share TweetNAIROBI, Kenya, Apr 20 – K..."
2,https://www.capitalfm.co.ke/news/2020/01/locus...,"13 SHARES Share Tweet\n\nMACHAKOS, Kenya, Jan ...","Locust invasion in Kitui, Machakos spells doom...",2020-01-28 15:11:26+00:00,"[residents, kitui, spells, town, locust, regio...","13 SHARES Share TweetMACHAKOS, Kenya, Jan 28 –..."
3,https://www.capitalfm.co.ke/news/2020/01/churc...,"5 SHARES Share Tweet\n\nNAIROBI, Kenya, Jan 26...",Church leader urges govt to partner with the w...,2020-01-27 05:26:48+00:00,"[swarms, govt, fighting, world, east, locust, ...","He, however, added that the locust invasion be..."
4,https://www.capitalfm.co.ke/news/2020/02/deser...,Locusts aggregate on the ground at Lerata vill...,Desert locusts invade Kisumu’s Muhoroni unsett...,2020-02-18 19:31:30+00:00,"[kisumus, old, desert, muhoroni, theyre, invad...",Locusts aggregate on the ground at Lerata vill...


In [10]:
df.tail()

,link,text,title,date,keywords,summary
22,https://www.capitalfm.co.ke/news/2020/01/locus...,"2 SHARES Share Tweet\n\nNAIROBI, Kenya, Jan 5 ...",Locust control operation well underway in Nort...,2020-01-05 15:14:30+00:00,"[desert, wajir, aerial, spread, locust, spray,...",Government Spokesman Cyrus Oguna says the aeri...
23,https://www.capitalfm.co.ke/news/2020/03/cs-mu...,"24 SHARES Share Tweet\n\nNAIROBI, Kenya, Mar 1...",CS Munya says egg-laying locusts largely inact...,2020-03-10 09:29:35+00:00,"[swarms, vegetation, capital, aerial, threat, ...","24 SHARES Share TweetNAIROBI, Kenya, Mar 10 – ..."
24,https://www.capitalfm.co.ke/news/2020/01/seen-...,"18 SHARES Share Tweet\n\nNAIROBI, Kenya, Jan 1...",Seen any insect resembling a locust? Send it t...,2020-01-10 11:34:44+00:00,"[grasshoppers, suspect, verification, locust, ...","Locusts have so far been sighted in Isiolo, Ma..."
25,https://www.capitalfm.co.ke/news/2020/02/senat...,0 SHARES Share Tweet\n\nMunya said a recent ae...,Senate team wants to question Munya on ‘aging’...,2020-02-19 05:30:00+00:00,"[yellow, question, aging, die, agriculture, fo...",Munya is required to face the Senate Committee...
26,https://www.capitalfm.co.ke/business/2020/02/g...,"0 SHARES Share Tweet\n\nNAIROBI, Kenya, Feb 15...",Govt says its working on measures to grow the ...,2020-02-15 11:44:50+00:00,"[economy, govt, measures, locust, working, tre...","0 SHARES Share TweetNAIROBI, Kenya, Feb 15 – T..."


In [ ]:
df.to_csv('n_capital-news.csv', index=False)